In [ ]:
!pip install hazm

     |████████████████████████████████| 317kB 15.3MB/s 
     |████████████████████████████████| 1.4MB 30.4MB/s 
     |████████████████████████████████| 235kB 43.9MB/s 
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394490 sha256=161c1cad8d24477d46957b7a692780ff2c1cddae1ca28543821d7a2af6f69015
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154253 sha256=1f19693084b8cbf978fbab85a616a50fda8be1714bfff5af17b811d7a60198f5
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


<p align='center'><strong>Imports</strong></p>

In [ ]:
import pandas as pd
import csv
import numpy as np
from sklearn.utils import shuffle
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from hazm import *

<p align='center'><strong>Mounting Google Drive</strong></p>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<p align='center'><strong>Reading English, French, Turkish and German, Persian and Arabic Dataset</strong></p>

In [ ]:
all_dataset = pd.read_csv('drive/MyDrive/Artificial Intelligence-Spring 2021/project/all.csv')
english_dataset = all_dataset.loc[all_dataset['Language'] == 'English'] 
german_dataset = all_dataset.loc[all_dataset['Language'] == 'German']
french_dataset = all_dataset.loc[all_dataset['Language'] == 'French']
turkish_dataset = all_dataset.loc[all_dataset['Language'] == 'Turkish']

language_dataset = [english_dataset, german_dataset, french_dataset, turkish_dataset]
result = pd.concat(language_dataset)


dic = {
    'Text': 'Id',
    'Language':'Category',
}
result = result.rename(columns=dic)


arabic_dataset = pd.read_csv('drive/MyDrive/Artificial Intelligence-Spring 2021/project/arabic.csv')
arabic_dataset = arabic_dataset.head(1800)
arabic_dataset.pop('targe')
arabic_dataset['Category'] = 'Arabic'
dic = {
    'text': 'Id',
    'Category': 'Category',
}
arabic_dataset = arabic_dataset.rename(columns=dic)

dataset_path = 'drive/MyDrive/Artificial Intelligence-Spring 2021/project/all_language_dataset.csv'

result.to_csv(dataset_path, mode='w', index=False)
arabic_dataset.to_csv(dataset_path, mode='a', header=False, index=False)

persian_dataset = pd.read_csv('drive/MyDrive/Artificial Intelligence-Spring 2021/project/persian.csv')
persian_dataset['Category'] = 'Persian'
persian_dataset.to_csv(dataset_path, mode='a', index=False, header='False')

all_language_dataset = pd.read_csv('drive/MyDrive/Artificial Intelligence-Spring 2021/project/all_language_dataset.csv')

# shuffling dataset
all_language_dataset = shuffle(all_language_dataset)

all_language_dataset

,Id,Category
2408,Le prix a été décerné à Jimmy Wales par David ...,French
4843,البرايم يستضيف لمنور وحماقي والداودية وباقبو و...,Arabic
271,"[14] It was founded on March 9, 2000, under th...",English
5517,پليس هند چهار تن از رهبران استقلال طلبان مسلم...,Persian
4619,البرنامج يبث مباشرة وموجه إلى كل أفراد الأسرة ...,Arabic
...,...,...
1646,nicht viel.,German
4592,المغني اللبناني قال إنه بصدد إعداد أغنية مغربي...,Arabic
1607,"Du kannst sagen, es ist mir egal, das ist so l...",German
30,The field of geology encompasses the study of ...,English


<p align='center'><strong>Deleting stopwords</strong></p>

In [ ]:
normalizer = Normalizer()

stopwords_list = []
stopwords_url = 'drive/MyDrive/Artificial Intelligence-Spring 2021/project/stopwords.txt'
f = open(stopwords_url, mode='r')
stopwords_list.append(f.read())
persian_stopwords = []
for i in stopwords_list:
  persian_stopwords.append(i)

In [ ]:
# with open('dataset.csv', 'w', newline='') as file:
#   writer = csv.writer(file)
#   writer.writerow(['Id', 'Category'])

# dataset = pd.read_csv('dataset.csv')

# creating a csv file for final dataset
# with open(dataset_path, mode='w', encoding='utf-8') as dataset_file:
#   wr = csv.writer(dataset_file, delimiter=',')
#   wr.writerow(['Id', 'Category'])
#   for (Id, Category) in enumerate(result):
#     wr.writerow((Id, Category))


In [ ]:
with open('drive/MyDrive/Artificial Intelligence-Spring 2021/project/persian.txt', 'r', encoding="utf-8") as f:
  lines = f.readlines()

with open('persiantext.csv', 'w', encoding="utf-8") as f:
  for line in lines:
      if not line.strip('\n').startswith("# File: www") \
          and not line.strip('\n').startswith("# Date") \
          and not line.strip('\n').startswith("# Headline"):
        f.write(line)

In [ ]:
print(len(all_language_dataset.loc[all_language_dataset['Category'] == 'French']))

1014


In [ ]:
categories = all_language_dataset[1].unique()
categories

array(['Turkish', 'English', 'Arabic', 'French', 'Persian', 'German',
       'Category'], dtype=object)

In [ ]:
for i in categories:
  print(i)

<p align='center'><strong>Vectorizing the Words.</strong></p>

In [ ]:
kaggle_task = pd.read_csv('drive/MyDrive/Artificial Intelligence-Spring 2021/project/task1.csv')
help_me = kaggle_task.Id

vectorizer = TfidfVectorizer(min_df=11, max_df=0.07, decode_error='strict', ngram_range=(1, 2), binary=0.5, sublinear_tf=True)
train_x = vectorizer.fit_transform(all_language_dataset.Id)
labelencoder = LabelEncoder()   
train_y = labelencoder.fit_transform(all_language_dataset.Category)

kaggle_task_x = vectorizer.transform(kaggle_task.Id)

<p align='center'><strong>Train-Test split test.</strong></p>

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.25, random_state=0)

<p align='center'><strong>Building Model</strong></p>

In [ ]:
def benchmark(clf):
  print("**", clf, "***")
  clf.fit(x_train, y_train)
  prediction = clf.predict(x_test)
  score = metrics.accuracy_score(y_test, prediction)
  print("accuracy:  %0.3f",  score)

results = []
clf = SGDClassifier(loss="modified_huber", max_iter=1500, shuffle=False)
name = "SGD Classifier"
print(name)
results.append(benchmark(clf))

SGD Classifier
** SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=1500, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=False, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False) ***
accuracy:  %0.3f 0.9811977715877437


<p align='center'><strong>Prediction on test data.</strong></p>

In [ ]:
kaggle_results = []
print("***  KAGGLE TEST  ***")
classifier = SGDClassifier(loss="modified_huber", max_iter=1500, shuffle=False)
classifier.fit(x_train, y_train)

kaggle_prediction = classifier.predict(kaggle_task_x)

***  KAGGLE TEST  ***


<p align='center'><strong>Writing results on the file</strong></p>

In [ ]:
preLabels = ['Id', 'Category']
labels_y = labelencoder.inverse_transform(kaggle_prediction)
labels_x = vectorizer.inverse_transform(kaggle_task_x)


result_file = 'drive/MyDrive/Artificial Intelligence-Spring 2021/project/result22.csv'
with open(result_file, mode='w') as result_file:
  wr = csv.writer(result_file, delimiter=',')
  wr.writerow(preLabels)
  for (id, item) in enumerate(labels_y, start=0):
    wr.writerow((help_me[id] , item))

In [ ]:
print(train_x[0])
y1 = np.zeros([train_x.shape[0], 6])
y1 = pd.DataFrame(y1)
y1

In [ ]:

def hypothesis(theta, X):
    return 1 / (1 + np.exp(-(np.dot(theta, X.T)))) - 0.0000001

def cost(X, y, theta):
    y1 = hypothesis(X, theta)
    return -(1/len(X)) * np.sum(y*np.log(y1) + (1-y)*np.log(1-y1))

def gradient_descent(X, y, theta, alpha, epochs):
    m = len(all_language_dataset)
    for i in range(0, epochs):
        for j in range(0, 10):
            theta = pd.DataFrame(theta)
            h = hypothesis(theta.iloc[:,j], X)
            for k in range(0, theta.shape[0]):
                theta.iloc[k, j] -= (alpha/m) * np.sum((h-y.iloc[:, j])*X.iloc[:, k])
            theta = pd.DataFrame(theta)
    return theta, cost


theta = np.zeros([train_x.shape[1]+1, y1.shape[1]])
theta = gradient_descent(train_x, y1, theta, 0.02, 1500)